In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
from scipy import signal
import sklearn as skl
%matplotlib tk

from ipywidgets import interact


plt.rcParams["figure.dpi"] = 150
plt.rcParams["figure.figsize"] = (5*(1+np.sqrt(5))/2,5)

In [16]:
raw = pd.read_csv("../mediciones/med6_4_16/13rps_int.csv", sep=",", comment="#")
data = raw.iloc[:,0:2]
data.X = data.iloc[:,0].astype('float64')
data.CH1 = data.iloc[:,1].astype('float64')
#data = np.loadtxt("../mediciones/med6_4_16/12rmp.csv",skiprows=2, delimiter=",")
#data[:,0] -= data[:,0].min()
#data[:,0] *= 1e-6
data = data.values

print("Frecuencia de muestreo: {}".format(1/np.diff(data[:,0])[0]))

Frecuencia de muestreo: 10000.0000000011


C:\Users\sadeus\Anaconda3\lib\site-packages\pandas\core\generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


### Obtención de la frecuencia fundamental

In [3]:
def get_freq(data):
    fft = np.fft.rfft(data[:,1])
    freq = np.fft.rfftfreq(data.shape[0], d = np.diff(data[:,0])[0])
    
    #Elimino el máximo inicial, que es un artefácto del algoritmo
    freq = freq[1:]
    fft = np.abs(fft[1:])
    
    #Obtengo el valor máximo, que va a ser la fundamental de la señal
    max_fft = fft == fft.max()
    
    #Ploteo de debuggeo
    #plt.plot(freq[freq < 200], fft[freq < 200],"bo-")
    #plt.plot(freq[max_fft], fft[max_fft],"ro")
    
    #Devuelvo la frecuencia, como float
    return freq[max_fft][0]

%matplotlib inline
get_freq(data)

12.81738281250141

### Ajuste de los datos por medio de filtros y gaussianas

In [13]:
#d = 1
#nmin = 1000 * d
#nmax = 1000 + d * 2000
T = data[:,0]
V = data[:,1]

#Obtengo automáticamente la frecuencia, a partir de FFT
f = get_freq(data)

filt = sp.signal.medfilt(V,kernel_size=25)
gauss = np.abs(sp.ndimage.gaussian_filter(filt, 50, order=1))
gauss = gauss/gauss.max()

fig = plt.figure(1)
plt.plot(T,V,'ro')
#Plot de debuggeo
#plt.plot(T,gauss,'bo')

peaks = sp.signal.find_peaks_cwt(gauss, np.array([30]))
plt.plot(T[peaks],V[peaks],'bd')
err_f = lambda x, a, b, c, d: a + b * sp.special.erf(np.sqrt(2)*(x - c) / d)



w = 2 * np.pi * f
R = 10
σ = []
DeltaT = 0.2 / f #Encontrar frecuencia automáticamente!
for i in peaks:
    ind = np.abs(T - T[i]) < DeltaT
    x = T[ind]
    y = V[ind]
    p0 = [y.max()/2, -y.max()/2, (x.max() + x.min())/2, 0.5*(x.max() - x.min())]
    try:
        p, cov = sp.optimize.curve_fit(err_f, x, y, p0 = p0)
        t = np.linspace(x.min(), x.max() , 1000)
        plt.plot(t, err_f(t,*p),figure=fig, linewidth=2)
        #print(p[3]*R*w)
        σ.append(p[3]*R*w)
    except RuntimeError:
        print("No se pudo ajustar")
        pass

        #A.append(mod_max(int(i + DeltaT/2)))
print(np.mean(np.array(σ)), np.sqrt(np.var(np.array(σ))))
print(np.sqrt(np.var(np.array(σ)))/np.mean(np.array(σ)))

2.25635054028 0.251611599088
0.111512637153
